# Backtest Results Analysis

This notebook provides tools for analyzing backtest results from the Kalshi Sports Alpha system.

## Contents
1. Setup and Imports
2. Load Backtest Results
3. Performance Metrics
4. Equity Curve Visualization
5. Trade Analysis


In [ ]:
# Setup and Imports
import sys
sys.path.insert(0, '..')

from datetime import datetime, timedelta
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np

# Import project modules
from backtest.simulator import BacktestSimulator, BacktestConfig, Position
from backtest.metrics import BacktestMetrics, calculate_metrics, calculate_drawdown
from backtest.scenarios import Scenario, ScenarioRunner, STANDARD_SCENARIOS

# Configure plotting
plt.style.use('seaborn-v0_8-whitegrid')
%matplotlib inline


## Create Sample Backtest Results

For demonstration, we create synthetic backtest results. In production, these would come from running the simulator.


In [ ]:
# Generate sample equity curve
np.random.seed(42)
initial_capital = 10000
n_days = 100

# Simulate returns with slight positive drift
daily_returns = np.random.normal(0.001, 0.02, n_days)
equity = initial_capital * np.cumprod(1 + daily_returns)

# Create equity curve data
dates = pd.date_range(start='2024-01-01', periods=n_days, freq='D')
equity_curve = [{'timestamp': d, 'equity': e} for d, e in zip(dates, equity)]

# Create sample positions
sample_positions = []
for i in range(30):
    entry_time = dates[np.random.randint(0, n_days-5)]
    exit_time = entry_time + timedelta(days=np.random.randint(1, 5))
    direction = np.random.choice(['YES', 'NO'])
    entry_price = np.random.uniform(0.3, 0.7)
    
    # Determine win/loss
    is_win = np.random.random() > 0.45  # 55% win rate
    if is_win:
        exit_price = 1.0 if direction == 'YES' else 0.0
    else:
        exit_price = 0.0 if direction == 'YES' else 1.0
    
    pos = Position(
        market_id=f"MKT{i:03d}",
        direction=direction,
        size=np.random.randint(10, 100),
        entry_price=entry_price,
        entry_time=entry_time,
        exit_price=exit_price,
        exit_time=exit_time,
    )
    sample_positions.append(pos)

print(f"Created equity curve with {len(equity_curve)} data points")
print(f"Created {len(sample_positions)} sample positions")


## Performance Metrics


In [ ]:
# Calculate comprehensive metrics
metrics = calculate_metrics(
    equity_curve=equity_curve,
    positions=sample_positions,
    initial_capital=initial_capital,
)

# Display metrics summary
print(metrics.summary())


## Equity Curve Visualization


In [ ]:
# Create equity curve DataFrame
equity_df = pd.DataFrame(equity_curve)
equity_df['timestamp'] = pd.to_datetime(equity_df['timestamp'])
equity_df.set_index('timestamp', inplace=True)

# Calculate drawdown
equity_values = equity_df['equity'].values
peak = np.maximum.accumulate(equity_values)
drawdown = (peak - equity_values) / peak

fig, axes = plt.subplots(3, 1, figsize=(14, 12))

# Equity curve
ax1 = axes[0]
ax1.plot(equity_df.index, equity_df['equity'], linewidth=2, color='steelblue')
ax1.axhline(y=initial_capital, color='gray', linestyle='--', alpha=0.7, label='Initial Capital')
ax1.fill_between(equity_df.index, initial_capital, equity_df['equity'], 
                  where=equity_df['equity'] >= initial_capital, alpha=0.3, color='green')
ax1.fill_between(equity_df.index, initial_capital, equity_df['equity'], 
                  where=equity_df['equity'] < initial_capital, alpha=0.3, color='red')
ax1.set_title('Equity Curve', fontsize=14)
ax1.set_ylabel('Portfolio Value ($)')
ax1.legend()
ax1.grid(True, alpha=0.3)

# Drawdown
ax2 = axes[1]
ax2.fill_between(equity_df.index, 0, -drawdown * 100, color='red', alpha=0.5)
ax2.set_title('Drawdown', fontsize=14)
ax2.set_ylabel('Drawdown (%)')
ax2.set_ylim(-max(drawdown) * 100 * 1.2, 1)
ax2.grid(True, alpha=0.3)

# Daily returns
daily_rets = equity_df['equity'].pct_change().dropna()
ax3 = axes[2]
colors = ['green' if r >= 0 else 'red' for r in daily_rets]
ax3.bar(daily_rets.index, daily_rets * 100, color=colors, alpha=0.7, width=1)
ax3.axhline(y=0, color='black', linewidth=0.5)
ax3.set_title('Daily Returns', fontsize=14)
ax3.set_ylabel('Return (%)')
ax3.set_xlabel('Date')
ax3.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()


## Trade Analysis
